## NLP - Clase 3 - UBA IA - Word2Vec


Por Pablo Casas



Objetivos:

Usar la lib mas utilizada de la industria para word2vec, crear modelos de skipgram y CBOW, con y sin negative sampling.

---

Librerías

In [1]:
%%capture 
# el %%capture hace que no aparezca mensaje de instalación
!pip install funpymodeling
from funpymodeling.exploratory import freq_tbl, status, profiling_num, cat_vars, num_vars, corr_pair
from funpymodeling.data_prep import todf

In [2]:
import pandas as pd
import seaborn as sns
from pandas_profiling import ProfileReport
from google.colab.data_table import DataTable
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import multiprocessing

from gensim.models import Word2Vec


In [4]:
pd.set_option('display.max_rows', 50)

### 1)  Lectura de datos

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
os.chdir('/content/drive/My Drive/fiuba/data')

In [7]:
!ls

datosPCA.csv  heart_disease.csv  __MACOSX  MNIST.zip  songs_dataset


In [8]:
df = pd.read_csv('songs_dataset/prince.txt', sep='/n', header=None)
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,0
0,All of this and more is for you
1,"With love, sincerity and deepest care"
2,My life with you I share
3,"Ever since I met you, baby"
4,I've been wantin' to lay you down


In [9]:
df.shape

(9616, 1)

## 3) Creación de modelos

In [10]:
df.loc[0,0]

'All of this and more is for you'

In [11]:
from keras.preprocessing.text import text_to_word_sequence

tokens=text_to_word_sequence(df.loc[0,0]) # df.loc[0,0], entran oraciones -> salen vectores de N posiciones (tokens)

tokens

['all', 'of', 'this', 'and', 'more', 'is', 'for', 'you']

In [12]:
df[:10]

,0
0,All of this and more is for you
1,"With love, sincerity and deepest care"
2,My life with you I share
3,"Ever since I met you, baby"
4,I've been wantin' to lay you down
5,But it's so hard to get you
6,"Baby, when you never come around"
7,Every day that you keep it away
8,It only makes me want it more
9,"Ooh baby, just say the word"


### Pequeña data prep

gensim acepta lista, de listas...

longitud variable 👏

In [13]:
l_sentence_tokens=[]
for _, row in df[:None].iterrows():
    l_sentence_tokens.append(text_to_word_sequence(row[0]))

In [14]:
l_sentence_tokens[:10]

[['all', 'of', 'this', 'and', 'more', 'is', 'for', 'you'],
 ['with', 'love', 'sincerity', 'and', 'deepest', 'care'],
 ['my', 'life', 'with', 'you', 'i', 'share'],
 ['ever', 'since', 'i', 'met', 'you', 'baby'],
 ["i've", 'been', "wantin'", 'to', 'lay', 'you', 'down'],
 ['but', "it's", 'so', 'hard', 'to', 'get', 'you'],
 ['baby', 'when', 'you', 'never', 'come', 'around'],
 ['every', 'day', 'that', 'you', 'keep', 'it', 'away'],
 ['it', 'only', 'makes', 'me', 'want', 'it', 'more'],
 ['ooh', 'baby', 'just', 'say', 'the', 'word']]

In [15]:
import multiprocessing

from gensim.models import Word2Vec

In [16]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer... google colab no se la juega
cores

2

In [17]:
# Paso 1
w2v_model = Word2Vec(min_count=3, # min cantidad ed palabras para incluir
                     window=2, # cant de palabras antes y desp de la predicha
                     size=300,  # dimensionalidad de los vectores 
                     workers=cores-1)

In [18]:
# Paso 2 / seria como el compile de keras
w2v_model.build_vocab(l_sentence_tokens, progress_per=10000)

In [19]:
# Paso 3
w2v_model.train(l_sentence_tokens, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(1284063, 1894530)

In [20]:
# cant de filas/casos/docs:
w2v_model.corpus_count

9616

In [21]:
# cant de palabras
len(w2v_model.wv.vocab)

1670

In [22]:
# palabras que mas se relacionan con...:
w2v_model.wv.most_similar(positive=["love"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('offer', 0.6453726291656494),
 ('machine', 0.6243289709091187),
 ('color', 0.6044940948486328),
 ('dear', 0.6011450290679932),
 ('loving', 0.5962963104248047),
 ('critics', 0.5930368900299072),
 ('because', 0.5822475552558899),
 ("9's", 0.5822150707244873),
 ('forever', 0.5776005983352661),
 ('life', 0.5775699615478516)]

`wv`: mapeo de palabras con embeddings

In [23]:
# palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('housequake', 0.17435520887374878),
 ("y'all", 0.15598955750465393),
 ('vous', 0.15314096212387085),
 ('an', 0.15122903883457184),
 ('tres', 0.14788272976875305),
 ('annie', 0.14745570719242096),
 ('zero', 0.14698533713817596),
 ('jam', 0.14661899209022522),
 ('etes', 0.13922348618507385),
 ('ship', 0.13467268645763397)]

In [24]:
#w2v_model.wv.most_similar(negative=["diedaa"]) #  no esta en el dic

#### ¿Entonces que palabras estan?

In [25]:
w2v_model.wv.vocab # no se puede acceder directamente....

{'all': <gensim.models.keyedvectors.Vocab at 0x7f119da7a438>,
 'of': <gensim.models.keyedvectors.Vocab at 0x7f119da7a198>,
 'this': <gensim.models.keyedvectors.Vocab at 0x7f119da7a7f0>,
 'and': <gensim.models.keyedvectors.Vocab at 0x7f119da7a358>,
 'more': <gensim.models.keyedvectors.Vocab at 0x7f119da7a400>,
 'is': <gensim.models.keyedvectors.Vocab at 0x7f11bb08d940>,
 'for': <gensim.models.keyedvectors.Vocab at 0x7f11bb08d9b0>,
 'you': <gensim.models.keyedvectors.Vocab at 0x7f116e469358>,
 'with': <gensim.models.keyedvectors.Vocab at 0x7f116e469400>,
 'love': <gensim.models.keyedvectors.Vocab at 0x7f116e4694a8>,
 'care': <gensim.models.keyedvectors.Vocab at 0x7f116e4695c0>,
 'my': <gensim.models.keyedvectors.Vocab at 0x7f116e469630>,
 'life': <gensim.models.keyedvectors.Vocab at 0x7f116e469710>,
 'i': <gensim.models.keyedvectors.Vocab at 0x7f116e469550>,
 'share': <gensim.models.keyedvectors.Vocab at 0x7f116e4696d8>,
 'ever': <gensim.models.keyedvectors.Vocab at 0x7f116e4692e8>,
 'si

In [26]:
# Parentesis: uso de dict:
list(w2v_model.wv.vocab.items())[:10]

[('all', <gensim.models.keyedvectors.Vocab at 0x7f119da7a438>),
 ('of', <gensim.models.keyedvectors.Vocab at 0x7f119da7a198>),
 ('this', <gensim.models.keyedvectors.Vocab at 0x7f119da7a7f0>),
 ('and', <gensim.models.keyedvectors.Vocab at 0x7f119da7a358>),
 ('more', <gensim.models.keyedvectors.Vocab at 0x7f119da7a400>),
 ('is', <gensim.models.keyedvectors.Vocab at 0x7f11bb08d940>),
 ('for', <gensim.models.keyedvectors.Vocab at 0x7f11bb08d9b0>),
 ('you', <gensim.models.keyedvectors.Vocab at 0x7f116e469358>),
 ('with', <gensim.models.keyedvectors.Vocab at 0x7f116e469400>),
 ('love', <gensim.models.keyedvectors.Vocab at 0x7f116e4694a8>)]

In [27]:
# Convertimos a algo mas amigable
w2c = dict()
for item in w2v_model.wv.vocab:
    w2c[item]=w2v_model.wv.vocab[item].count

In [28]:
list(w2c.items())[0:10]

[('all', 575),
 ('of', 469),
 ('this', 362),
 ('and', 1050),
 ('more', 106),
 ('is', 508),
 ('for', 93),
 ('you', 560),
 ('with', 311),
 ('love', 591)]

In [29]:
# Ordenamos:
w2c_ordered = sorted(w2c.items(), key=lambda x: x[1], reverse=True)

In [30]:
w2c_ordered

[('the', 2221),
 ('u', 2057),
 ('i', 1920),
 ('a', 1186),
 ('2', 1107),
 ('and', 1050),
 ('me', 896),
 ('it', 820),
 ('in', 810),
 ('your', 628),
 ('that', 604),
 ('my', 593),
 ('love', 591),
 ('do', 576),
 ('all', 575),
 ('we', 571),
 ('you', 560),
 ('on', 543),
 ('yeah', 532),
 ('baby', 526),
 ('be', 525),
 ('is', 508),
 ('of', 469),
 ("don't", 458),
 ('to', 454),
 ('4', 440),
 ("i'm", 437),
 ('so', 399),
 ('if', 394),
 ('go', 387),
 ('come', 374),
 ('just', 368),
 ('oh', 368),
 ('this', 362),
 ('what', 312),
 ('with', 311),
 ('got', 309),
 ('no', 308),
 ('when', 306),
 ('can', 298),
 ('get', 295),
 ('like', 295),
 ('say', 291),
 ('will', 283),
 ('wanna', 281),
 ('know', 274),
 ("it's", 261),
 ('up', 260),
 ('time', 257),
 ('but', 249),
 ('want', 237),
 ('gonna', 232),
 ('down', 224),
 ('la', 214),
 ('now', 209),
 ('are', 187),
 ("can't", 185),
 ('was', 184),
 ("i'll", 178),
 ('see', 175),
 ('they', 174),
 ('there', 165),
 ('how', 160),
 ('tell', 159),
 ("that's", 157),
 ('have', 155

In [31]:
w2v_model.wv.most_similar(positive=["baby"], topn=5)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('marvelous', 0.7242048382759094),
 ('sweet', 0.6749805212020874),
 ('help', 0.6487023830413818),
 ('ooo', 0.6423391103744507),
 ('teach', 0.6352441310882568)]

## Probemos otro texto, nietzsche!



In [32]:
df2 = pd.read_csv('https://s3.amazonaws.com/text-datasets/nietzsche.txt', sep='/n', header=None)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [33]:
l_sentence_tokens2=[]
for _, row in df2[:None].iterrows():
    l_sentence_tokens2.append(text_to_word_sequence(row[0]))

In [34]:
l_sentence_tokens2

[['preface'],
 ['supposing',
  'that',
  'truth',
  'is',
  'a',
  'woman',
  'what',
  'then',
  'is',
  'there',
  'not',
  'ground'],
 ['for',
  'suspecting',
  'that',
  'all',
  'philosophers',
  'in',
  'so',
  'far',
  'as',
  'they',
  'have',
  'been'],
 ['dogmatists',
  'have',
  'failed',
  'to',
  'understand',
  'women',
  'that',
  'the',
  'terrible'],
 ['seriousness',
  'and',
  'clumsy',
  'importunity',
  'with',
  'which',
  'they',
  'have',
  'usually',
  'paid'],
 ['their',
  'addresses',
  'to',
  'truth',
  'have',
  'been',
  'unskilled',
  'and',
  'unseemly',
  'methods',
  'for'],
 ['winning',
  'a',
  'woman',
  'certainly',
  'she',
  'has',
  'never',
  'allowed',
  'herself',
  'to',
  'be',
  'won',
  'and'],
 ['at',
  'present',
  'every',
  'kind',
  'of',
  'dogma',
  'stands',
  'with',
  'sad',
  'and',
  'discouraged',
  'mien',
  'if'],
 ['indeed',
  'it',
  'stands',
  'at',
  'all',
  'for',
  'there',
  'are',
  'scoffers',
  'who',
  'maintai

### En clase vimos: CBOW, Skip-gram

`sg = 0` CBOW (por defecto), 1 para skip gram, negative sampling....

In [35]:
w2v_model2 = Word2Vec(min_count=5,
                     window=2,
                     size=300, 
                     negative=20, # cantidad de negative samples... 0 es no se usa
                     workers=cores-1, 
                     sg=1) # modelo skipgram

In [36]:
w2v_model2.build_vocab(l_sentence_tokens2, progress_per=10000)

In [37]:
w2v_model2.train(l_sentence_tokens2, total_examples=w2v_model2.corpus_count, epochs=30)

(1783394, 3040740)

In [38]:
print(w2v_model2.corpus_count)
print(len(w2v_model2.wv.vocab))

9037
2170


In [39]:
w2v_model2.wv.most_similar(positive=["god"], topn=10)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('figure', 0.5852356553077698),
 ('invent', 0.5833888053894043),
 ('saints', 0.5699678063392639),
 ('greeks', 0.5636512041091919),
 ('pessimist', 0.5606003403663635),
 ('fast', 0.5592543482780457),
 ('lacks', 0.5587148666381836),
 ('benevolence', 0.5573039054870605),
 ('book', 0.5541856288909912),
 ('servants', 0.5536471605300903)]

In [40]:
w2v_model2.wv.most_similar(positive=["however"], topn=10)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('asked', 0.6304600238800049),
 ('appearances', 0.6080811619758606),
 ('homer', 0.5886005163192749),
 ('notwithstanding', 0.5779083967208862),
 ('eventually', 0.5777988433837891),
 ('talk', 0.5777338743209839),
 ('doing', 0.5744763612747192),
 ('learnt', 0.5727075338363647),
 ('remarkable', 0.5710222721099854),
 ('demand', 0.5706275701522827)]

### "Tuning del modelo" 

Definimos el callback ([Fuente](https://stackoverflow.com/questions/52038651/loss-does-not-decrease-during-training-word2vec-gensim))

In [41]:
from gensim.models.callbacks import CallbackAny2Vec
# init callback class 
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [42]:
w2v_model2 = Word2Vec(min_count=5,
                     window=2,
                     size=300, 
                     negative=20,
                     workers=cores-1, 
                     sg=1)

In [43]:
w2v_model2.build_vocab(l_sentence_tokens2, progress_per=10000)

In [44]:
# Agregamos el compute_loss y callbacks
w2v_model2.train(l_sentence_tokens2, total_examples=w2v_model2.corpus_count, epochs=20,# report_delay=1, 
                    compute_loss = True, callbacks=[callback()])

Loss after epoch 0: 785671.0625
Loss after epoch 1: 548457.4375
Loss after epoch 2: 507018.125
Loss after epoch 3: 472614.875
Loss after epoch 4: 435342.25
Loss after epoch 5: 434696.0
Loss after epoch 6: 433183.25
Loss after epoch 7: 429719.0
Loss after epoch 8: 416801.0
Loss after epoch 9: 406394.0
Loss after epoch 10: 406157.5
Loss after epoch 11: 403562.0
Loss after epoch 12: 401569.0
Loss after epoch 13: 400544.5
Loss after epoch 14: 398555.5
Loss after epoch 15: 396163.0
Loss after epoch 16: 393853.0
Loss after epoch 17: 394158.0
Loss after epoch 18: 392144.5
Loss after epoch 19: 383717.0


(1188765, 2027160)

#### Probamos otras ventanas

Window size: for skip-gram usually around 10, for CBOW around 5 (Fuente: https://code.google.com/archive/p/word2vec/)

In [45]:
w2v_model2 = Word2Vec(min_count=5,
                     window=10,
                     size=300, 
                     negative=20,
                     workers=cores-1, 
                     sg=1)

In [46]:
w2v_model2.build_vocab(l_sentence_tokens2, progress_per=10000)

In [47]:
# agregamos el compute_loss y callbacks
w2v_model2.train(l_sentence_tokens2, total_examples=w2v_model2.corpus_count, epochs=20,
                    compute_loss = True, callbacks=[callback()])

Loss after epoch 0: 1345148.5
Loss after epoch 1: 1027961.0
Loss after epoch 2: 939037.25
Loss after epoch 3: 936954.75
Loss after epoch 4: 887046.5
Loss after epoch 5: 877026.5
Loss after epoch 6: 874312.0
Loss after epoch 7: 868888.5
Loss after epoch 8: 866442.0
Loss after epoch 9: 846720.0
Loss after epoch 10: 843638.0
Loss after epoch 11: 836440.0
Loss after epoch 12: 832538.0
Loss after epoch 13: 831929.0
Loss after epoch 14: 826842.0
Loss after epoch 15: 821995.0
Loss after epoch 16: 823165.0
Loss after epoch 17: 819611.0
Loss after epoch 18: 813853.0
Loss after epoch 19: 805854.0


(1188627, 2027160)

In [48]:
w2v_model2.get_latest_training_loss()

17725402.0

In [49]:
# hmmm....  a ver una pruba mas:

In [50]:
w2v_model2 = Word2Vec(min_count=5,
                     window=1,
                     size=300, 
                     negative=20,
                     workers=cores-1, 
                     sg=1)

In [51]:
w2v_model2.build_vocab(l_sentence_tokens2, progress_per=10000)
w2v_model2.train(l_sentence_tokens2, total_examples=w2v_model2.corpus_count, epochs=20,
                    compute_loss = True, callbacks=[callback()])

Loss after epoch 0: 623887.75
Loss after epoch 1: 422399.5
Loss after epoch 2: 361051.125
Loss after epoch 3: 357366.625
Loss after epoch 4: 350173.5
Loss after epoch 5: 304712.0
Loss after epoch 6: 302415.0
Loss after epoch 7: 302544.25
Loss after epoch 8: 300905.75
Loss after epoch 9: 298885.5
Loss after epoch 10: 297933.5
Loss after epoch 11: 295280.0
Loss after epoch 12: 280649.0
Loss after epoch 13: 277850.0
Loss after epoch 14: 277900.5
Loss after epoch 15: 276289.0
Loss after epoch 16: 274365.0
Loss after epoch 17: 274177.0
Loss after epoch 18: 272298.0
Loss after epoch 19: 272827.0


(1188954, 2027160)

¿Cuál es el mejor modelo para estos datos?
<br>
<br>
<br>
Nota: ojo hay un bug/discrepa con el registro de la loss: https://github.com/RaRe-Technologies/gensim/issues/2735

## Visualizar resultados!

[Fuente código](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html), (el grafico no anda).


In [52]:
vector = w2v_model2.wv['greeks']

vector.shape

(300,)

In [53]:
# Embeddings de todas las palabras:
w2v_model2.wv.vectors.shape

(2170, 300)

In [54]:
len(list(w2v_model2.wv.vocab)) # cant de palabras del corpus

2170

In [55]:
w2v_model2.wv.index2word # palabras "texto" del corpus

['the',
 'of',
 'and',
 'to',
 'in',
 'is',
 'a',
 'that',
 'it',
 'as',
 'for',
 'which',
 'be',
 'not',
 'with',
 'he',
 'all',
 'this',
 'are',
 'his',
 'one',
 'has',
 'or',
 'an',
 'but',
 'have',
 'at',
 'we',
 'they',
 'who',
 'man',
 'their',
 'from',
 'more',
 'will',
 'by',
 'so',
 'its',
 'there',
 'what',
 'on',
 'i',
 'such',
 'even',
 'only',
 'our',
 'no',
 'himself',
 'them',
 'itself',
 'most',
 'when',
 'if',
 'was',
 'him',
 'every',
 'been',
 'men',
 'also',
 'must',
 'would',
 'good',
 'us',
 'may',
 'perhaps',
 'can',
 'how',
 'upon',
 'world',
 'than',
 'life',
 'does',
 'self',
 'still',
 'other',
 'things',
 'into',
 'very',
 'do',
 'nature',
 'own',
 'any',
 'something',
 'like',
 'always',
 'these',
 'out',
 'should',
 'too',
 'great',
 'much',
 'were',
 'time',
 'soul',
 'themselves',
 'without',
 'now',
 'had',
 'being',
 'new',
 'some',
 'same',
 'spirit',
 'love',
 'morality',
 'say',
 'moral',
 'those',
 'about',
 'means',
 'god',
 'you',
 'because',
 'h

In [56]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [57]:
# REDUCCION DE DIM
x_vals, y_vals, labels = reduce_dimensions(w2v_model2)

In [58]:
# GRAFICAMOS CON PLOTLY
import plotly.graph_objects as go
import plotly.express as px

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])

fig.show(renderer="colab") # esto para plotly en colab

## Ejercicio, crear el propio word2vec!

1) Para el dataset dado, si se animan concatenar todo en una sola linea, sino usar lo mostrado en clase

2) Crear un modelo skipgram, otro CBOW, quedarse con el que menor loss tenga a lo largo de 20 epochs. 

2bis, si quieren optimizar el tamaño de la ventana también sería interesante para este caso.

3) Para googlear (fácil): encontrar la función de gensim que, dada dos palabras, nos devuelve cuanto se parecen

4) Para googlear (fácil): cargar fasttext e imprimir con plotly palabras al azar. 
4bis) Comparar algunas palabras con el texto de Nietzche

In [59]:
df = pd.read_csv('songs_dataset/bob-dylan.txt', sep='/n', header=None)
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0
0,"“There must be some way out of here,” said the..."
1,"“There’s too much confusion, I can’t get no re..."
2,"Businessmen, they drink my wine, plowmen dig m..."
3,None of them along the line know what any of i...
4,"“No reason to get excited,” the thief, he kind..."


In [60]:
df.loc[0,0]

'“There must be some way out of here,” said the joker to the thief'

In [61]:
from keras.preprocessing.text import text_to_word_sequence

tokens=text_to_word_sequence(df.loc[0,0]) # df.loc[0,0], entran oraciones -> salen vectores de N posiciones (tokens)
tokens

['“there',
 'must',
 'be',
 'some',
 'way',
 'out',
 'of',
 'here',
 '”',
 'said',
 'the',
 'joker',
 'to',
 'the',
 'thief']

#### Punto 1

In [62]:
l_sentence_tokens=[[]]
for _, row in df[:None].iterrows():
    #l_sentence_tokens.append(text_to_word_sequence(row[0]))
    l_sentence_tokens[0].extend(text_to_word_sequence(row[0]))
l_sentence_tokens    

[['“there',
  'must',
  'be',
  'some',
  'way',
  'out',
  'of',
  'here',
  '”',
  'said',
  'the',
  'joker',
  'to',
  'the',
  'thief',
  '“there’s',
  'too',
  'much',
  'confusion',
  'i',
  'can’t',
  'get',
  'no',
  'relief',
  'businessmen',
  'they',
  'drink',
  'my',
  'wine',
  'plowmen',
  'dig',
  'my',
  'earth',
  'none',
  'of',
  'them',
  'along',
  'the',
  'line',
  'know',
  'what',
  'any',
  'of',
  'it',
  'is',
  'worth”',
  '“no',
  'reason',
  'to',
  'get',
  'excited',
  '”',
  'the',
  'thief',
  'he',
  'kindly',
  'spoke',
  '“there',
  'are',
  'many',
  'here',
  'among',
  'us',
  'who',
  'feel',
  'that',
  'life',
  'is',
  'but',
  'a',
  'joke',
  'but',
  'you',
  'and',
  'i',
  'we’ve',
  'been',
  'through',
  'that',
  'and',
  'this',
  'is',
  'not',
  'our',
  'fate',
  'so',
  'let',
  'us',
  'not',
  'talk',
  'falsely',
  'now',
  'the',
  'hour',
  'is',
  'getting',
  'late”',
  'all',
  'along',
  'the',
  'watchtower',
  'prin

#### Punto 2

##### CBOW

In [63]:
# CBOW (sg=0)

# Paso 1
w2v_model_cbow = Word2Vec(min_count=3, # min cantidad ed palabras para incluir
                     window=2, # cant de palabras antes y desp de la predicha
                     size=300,  # dimensionalidad de los vectores 
                     workers=cores-1,
                     sg=0)
# Paso 2 / seria como el compile de keras
w2v_model_cbow.build_vocab(l_sentence_tokens, progress_per=10000)

# Paso 3
w2v_model_cbow.train(l_sentence_tokens, total_examples=w2v_model_cbow.corpus_count,
                     epochs=20, report_delay=1,compute_loss = True, 
                     callbacks=[callback()])

# cant de filas/casos/docs:
print("Corpus count:", w2v_model.corpus_count)

# cant de palabras
print("Vocab count:", len(w2v_model.wv.vocab))

Loss after epoch 0: 42324.3359375
Loss after epoch 1: 42258.546875
Loss after epoch 2: 40357.3203125
Loss after epoch 3: 35160.890625
Loss after epoch 4: 32028.015625
Loss after epoch 5: 30386.15625
Loss after epoch 6: 29280.90625
Loss after epoch 7: 28650.609375
Loss after epoch 8: 28263.96875
Loss after epoch 9: 27874.65625
Loss after epoch 10: 27684.59375
Loss after epoch 11: 27532.125
Loss after epoch 12: 27425.75
Loss after epoch 13: 27308.40625
Loss after epoch 14: 27191.90625
Loss after epoch 15: 27172.53125
Loss after epoch 16: 27124.09375
Loss after epoch 17: 27112.1875
Loss after epoch 18: 27003.6875
Loss after epoch 19: 27009.75
Corpus count: 9616
Vocab count: 1670


In [64]:
# palabras que mas se relacionan con...:
w2v_model_cbow.wv.most_similar(positive=["thief"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning:

Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.



[('their', 0.9997637271881104),
 ('hands', 0.9997622966766357),
 ('made', 0.9997611045837402),
 ('were', 0.9997556209564209),
 ('i’d', 0.9997555017471313),
 ('before', 0.9997555017471313),
 ('past', 0.9997549057006836),
 ('about', 0.9997501373291016),
 ('i’ve', 0.9997493028640747),
 ("i'm", 0.9997389316558838)]

##### Skipgram


In [65]:
# SkipGram (sg=1)

# Paso 1
w2v_model_skipgram = Word2Vec(min_count=3, # min cantidad ed palabras para incluir
                     window=2, # cant de palabras antes y desp de la predicha
                     size=300,  # dimensionalidad de los vectores 
                     workers=cores-1,
                     sg=1)
# Paso 2 / seria como el compile de keras
w2v_model_skipgram.build_vocab(l_sentence_tokens, progress_per=10000)

# Paso 3
w2v_model_skipgram.train(l_sentence_tokens, total_examples=w2v_model_skipgram.corpus_count,
                     epochs=20, report_delay=1,compute_loss = True, 
                     callbacks=[callback()])


# cant de filas/casos/docs:
print("Corpus count:", w2v_model_skipgram.corpus_count)

# cant de palabras
print("Vocab count:", len(w2v_model_skipgram.wv.vocab))

Loss after epoch 0: 126810.1015625
Loss after epoch 1: 116399.4609375
Loss after epoch 2: 96869.9375
Loss after epoch 3: 87891.59375
Loss after epoch 4: 83473.4375
Loss after epoch 5: 82726.71875
Loss after epoch 6: 81767.6875
Loss after epoch 7: 80465.5625
Loss after epoch 8: 80074.1875
Loss after epoch 9: 80005.75
Loss after epoch 10: 78577.625
Loss after epoch 11: 78435.5625
Loss after epoch 12: 77709.25
Loss after epoch 13: 76994.875
Loss after epoch 14: 76890.625
Loss after epoch 15: 76804.0
Loss after epoch 16: 76627.375
Loss after epoch 17: 76119.25
Loss after epoch 18: 75999.25
Loss after epoch 19: 75839.125
Corpus count: 1
Vocab count: 1451


In [66]:
# palabras que mas se relacionan con...:
w2v_model_skipgram.wv.most_similar(positive=["love"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning:

Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.



[('stand', 0.985895574092865),
 ('even', 0.9858118295669556),
 ('later', 0.9846743941307068),
 ('true', 0.9842417240142822),
 ('talk', 0.9842323064804077),
 ('should', 0.9838076829910278),
 ('makes', 0.9837489724159241),
 ('milk', 0.9836753606796265),
 ('great', 0.9834966659545898),
 ('plain', 0.9833266735076904)]

##### Conclusiones

- CBOW tiene menor error para el tamaño de ventana elegido:
  - CBOW Loss after epoch 19: 27012.75
  - Skipgram Loss after epoch 19: 75836.25

#### Punto 3

In [67]:
w2v_model_skipgram.wv.wmdistance(["love"],["true"])

0.28456562757492065

In [68]:
w2v_model_skipgram.wv.wmdistance(["every"],["love"])

0.3307538330554962

#### Punto 4

Para googlear (fácil): cargar fasttext e imprimir con plotly palabras al azar. 4bis) Comparar algunas palabras con el texto de Nietzche


In [70]:
!pip install fasttext

     |████████████████████████████████| 71kB 3.3MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3038331 sha256=29fb7cd11c24264f5a3314a88bd5469d4bd8e79008ce9cb7bea8923fa653028e
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [73]:
!wget 'https://s3.amazonaws.com/text-datasets/nietzsche.txt'

--2020-11-20 02:31:14--  https://s3.amazonaws.com/text-datasets/nietzsche.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.107.166
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.107.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 600901 (587K) [text/plain]
Saving to: ‘nietzsche.txt’

nietzsche.txt       100%[===================>] 586.82K  --.-KB/s    in 0.07s   

2020-11-20 02:31:15 (7.89 MB/s) - ‘nietzsche.txt’ saved [600901/600901]



In [75]:
import fasttext
model = fasttext.train_unsupervised("nietzsche.txt", minn=2, maxn=5, dim=300)

In [80]:
help(model)

Help on _FastText in module fasttext.FastText object:

class _FastText(builtins.object)
 |  This class defines the API to inspect models and should not be used to
 |  create objects. It will be returned by functions such as load_model or
 |  train.
 |  
 |  In general this API assumes to be given only unicode for Python2 and the
 |  Python3 equvalent called str for any string-like arguments. All unicode
 |  strings are then encoded as UTF-8 and fed to the fastText C++ API.
 |  
 |  Methods defined here:
 |  
 |  __contains__(self, word)
 |  
 |  __getitem__(self, word)
 |  
 |  __init__(self, model_path=None, args=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  get_analogies(self, wordA, wordB, wordC, k=10, on_unicode_error='strict')
 |  
 |  get_dimension(self)
 |      Get the dimension (size) of a lookup vector (hidden layer).
 |  
 |  get_input_matrix(self)
 |      Get a reference to the full input matrix of a Model. This only
 |      works if t

(100, 300)

In [105]:
input_matrix = model.get_input_matrix()
n_words = 100
random_words = np.random.choice(model.get_words(),n_words)
vector_id_list = [model.get_word_id(word) for word in random_words]
vectors = input_matrix[vector_id_list]
print(vectors.shape)

num_dimensions = 2  
tsne = TSNE(n_components=num_dimensions, random_state=0)
vectors = tsne.fit_transform(vectors)
x_vals = [v[0] for v in vectors]
y_vals = [v[1] for v in vectors]

# GRAFICAMOS CON PLOTLY
import plotly.graph_objects as go
import plotly.express as px

MAX_WORDS=200
fig = px.scatter(x=x_vals, y=y_vals, text=random_words)

fig.show(renderer="colab") # esto para plotly en colab

(100, 300)


In [108]:
model.get_nearest_neighbors('god', k=10)

[(0.9999932646751404, 'good'),
 (0.999992311000824, 'Good'),
 (0.9999919533729553, '"good'),
 (0.999991238117218, 'food'),
 (0.9999903440475464, 'laid'),
 (0.9999901056289673, 'bad'),
 (0.99998939037323, 'blood'),
 (0.9999886751174927, 'rid'),
 (0.9999886751174927, 'old'),
 (0.9999871850013733, 'God')]

In [109]:
model.get_nearest_neighbors('truth', k=10)

[(0.9999967813491821, 'truth.'),
 (0.9999958872795105, 'truths.'),
 (0.999995768070221, 'truth,'),
 (0.9999955296516418, '"truth"'),
 (0.9999942183494568, 'teeth'),
 (0.9999937415122986, 'truths'),
 (0.9999919533729553, 'author'),
 (0.9999917149543762, 'through'),
 (0.9999908804893494, 'tragedy'),
 (0.9999907612800598, 'triumph')]

In [110]:
model.get_nearest_neighbors('love', k=10)

[(0.9999896287918091, 'gave'),
 (0.9999893307685852, 'Love'),
 (0.9999885559082031, 'above'),
 (0.9999870657920837, 'loved'),
 (0.999986469745636, 'live'),
 (0.9999852180480957, 'like'),
 (0.9999845623970032, 'noble'),
 (0.9999845027923584, 'trouble'),
 (0.9999845027923584, 'look'),
 (0.9999838471412659, '"We')]

In [117]:
model.get_analogies('man','','', k=10)

[(0.999989926815033, 'human'),
 (0.9999871253967285, 'man,'),
 (0.9999871253967285, 'Roman'),
 (0.9999862909317017, 'German'),
 (0.9999858736991882, 'Man'),
 (0.9999855756759644, 'Christian'),
 (0.9999850988388062, 'mankind,'),
 (0.9999842047691345, 'Woman'),
 (0.9999839663505554, 'making'),
 (0.9999839067459106, 'modern')]

In [118]:
model.get_analogies('life','','', k=10)

[(0.9999963045120239, 'life:'),
 (0.9999932646751404, 'life.'),
 (0.9999921321868896, 'assume'),
 (0.9999918937683105, 'acquire'),
 (0.9999915957450867, 'mediocre'),
 (0.9999914765357971, 'alive'),
 (0.9999914169311523, 'Greek'),
 (0.9999914169311523, 'aware'),
 (0.9999905824661255, 'rule'),
 (0.9999905228614807, 'advantage')]

In [119]:
model.get_analogies('path','','', k=10)

[(0.9999932646751404, 'death'),
 (0.9999901056289673, 'great,'),
 (0.9999899864196777, 'greater,'),
 (0.999989926815033, 'beneath'),
 (0.9999896287918091, 'matter,'),
 (0.9999895691871643, 'henceforth'),
 (0.9999894499778748, 'stone'),
 (0.9999889731407166, 'manner,'),
 (0.9999889135360718, 'depth'),
 (0.9999887943267822, 'sympathetic')]